In [1]:
import pandas as pd
import torch as tc
import numpy as np
import multiprocessing
import re
from transformers import DistilBertTokenizerFast
import torch
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
# from pyevmasm import disassemble_hex
# from evmdasm import EvmBytecode



/home/venglov/.venv/main/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'pyevmasm'

In [16]:
malicious_df = pd.read_csv('../data_v2/malicious_smart_contracts.csv')

In [17]:
malicious_df

,contract_address,contract_tag,contract_creator,contract_creation_tx,contract_creator_tag,source,notes,contract_creator_etherscan_label
0,0x04ae3226c80e8c04d35e6e56089345bdd06da6de,Multichain Exploiter 10,0x98f5b0a99f258a09403d965647ee9d53677914af,0xe962b06db95db1dedb8b1664bf93bb8816c0cfbd1876...,NaN,Luabase ethereum.tags table,NaN,exploit
1,0xc5ac25cfc2b8284e84ca47dad21cf1319f732c11,NaN,0x07e02088d68229300ae503395c6536f09179dc3e,0xae03b08f220e5d7750e97e67ea85ac6e69649f6fd6d8...,xToken Exploiter,Luabase ethereum.tags table,NaN,heist
2,0x79dbe9bbde91a35fa8148a14084979a531fe57ea,NaN,0x11112f684cb88d43ca0e132e585e882606063fbe,0x07766fa0cb09dc5221763049c4a2fbbf3556c8660490...,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,NaN,heist
3,0x3b1ea5b11d12452693f9bd290ac2100394e6850f,NaN,0x11112f684cb88d43ca0e132e585e882606063fbe,0x10c93b07d287419254869c45c2323df43110cfd29443...,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,NaN,heist
4,0x682dcf2f4a6e46c222927a54529b4965fb313bf2,NaN,0x11112f684cb88d43ca0e132e585e882606063fbe,0x961f42e3a9986fb1ab15f2bc36434cb9e0d6739bf331...,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,NaN,heist
...,...,...,...,...,...,...,...,...
748,0x2df9c154fe24d081cfe568645fb4075d725431e0,NaN,0xd3127a793daf3aa9424525e893b494d17e798d39,0x34d996b92754dc3895a52bca07f6785d62d161f95b36...,NaN,https://github.com/SunWeb3Sec/DeFiHackLabs,20221011 Templedao - Insufficient access control,NaN
749,0x9682f31b3f572988f93c2b8382586ca26a866475,Rabby Swap Approval Exploiter,0xb687550842a24d7fbc6aad238fd7e0687ed59d55,0xbab47d1b4f99d5e61daa922414301d820a5b9c483a6f...,NaN,https://github.com/SunWeb3Sec/DeFiHackLabs,20221011 Rabby Wallet SwapRouter - Arbitrary E...,exploit
750,0x140cca423081ed0366765f18fc9f5ed299699388,Earning.Farm Exploiter,0xdf31f4c8dc9548eb4c416af26dc396a25fde4d5f,0x99c760442aecf00d46ecad7739f7935c41d76b2f8502...,NaN,https://github.com/SunWeb3Sec/DeFiHackLabs,20221014 EFLeverVault - Verify flashLoan Callback,exploit
751,0xfd4dccd754eaaa8c9196998c5bb06a56df6a1d95,NaN,0xcc1a341d0f2a06eaba436935399793f05c2bbe92,0xb71437e21a039eede08e5ad11bb9ba8b475dff4b5c2c...,NaN,https://github.com/SunWeb3Sec/DeFiHackLabs,20221017 Uerii Token - Access Control,NaN


In [18]:
heist_df = malicious_df.loc[malicious_df.contract_creator_etherscan_label == 'heist']

In [19]:
exploit_df = malicious_df.loc[malicious_df.contract_creator_etherscan_label == 'exploit']

In [20]:
sc_df = pd.concat([heist_df, exploit_df]).reset_index(drop=True)

In [21]:
sc_df

,contract_address,contract_tag,contract_creator,contract_creation_tx,contract_creator_tag,source,notes,contract_creator_etherscan_label
0,0xc5ac25cfc2b8284e84ca47dad21cf1319f732c11,NaN,0x07e02088d68229300ae503395c6536f09179dc3e,0xae03b08f220e5d7750e97e67ea85ac6e69649f6fd6d8...,xToken Exploiter,Luabase ethereum.tags table,NaN,heist
1,0x79dbe9bbde91a35fa8148a14084979a531fe57ea,NaN,0x11112f684cb88d43ca0e132e585e882606063fbe,0x07766fa0cb09dc5221763049c4a2fbbf3556c8660490...,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,NaN,heist
2,0x3b1ea5b11d12452693f9bd290ac2100394e6850f,NaN,0x11112f684cb88d43ca0e132e585e882606063fbe,0x10c93b07d287419254869c45c2323df43110cfd29443...,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,NaN,heist
3,0x682dcf2f4a6e46c222927a54529b4965fb313bf2,NaN,0x11112f684cb88d43ca0e132e585e882606063fbe,0x961f42e3a9986fb1ab15f2bc36434cb9e0d6739bf331...,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,NaN,heist
4,0xbc48cd3265fd6d6cd413cd3e7082c27993baf8b2,NaN,0x11112f684cb88d43ca0e132e585e882606063fbe,0x9901c2669d7d00079e8f84aec48ed03c620d50077231...,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,NaN,heist
...,...,...,...,...,...,...,...,...
111,0x016e20cad525bfb802ae3f12ecbcb64b5ad8410b,NaN,0xfa2731d0bede684993ab1109db7ecf5bf33e8051,0xa078f866085f7009ae7f097dbaa5ee688ff122aacfb2...,Multichain Exploiter 4 (Whitehat),Luabase ethereum.tags table,NaN,exploit
112,0xb5c827fdbbee6f6e9df3a5cb499aedf5927de1b8,Multichain Exploiter 6,0xfa2731d0bede684993ab1109db7ecf5bf33e8051,0x71bc04a3c444c480f18fbe30835b5bd0032a59b9dce0...,NaN,Luabase ethereum.tags table,NaN,exploit
113,0xe167cdaac8718b90c03cf2cb75dc976e24ee86d3,XaveFinance Exploit Contract,0x0f44f3489d17e42ab13a6beb76e57813081fc1e2,0x71e2e18665d16bb2064bff4ae4c6e5a8bf366721e511...,XaveFinance Exploiter,https://github.com/SunWeb3Sec/DeFiHackLabs,20221009 Xave Finance - Malicious Proposal Min...,exploit
114,0x9682f31b3f572988f93c2b8382586ca26a866475,Rabby Swap Approval Exploiter,0xb687550842a24d7fbc6aad238fd7e0687ed59d55,0xbab47d1b4f99d5e61daa922414301d820a5b9c483a6f...,NaN,https://github.com/SunWeb3Sec/DeFiHackLabs,20221011 Rabby Wallet SwapRouter - Arbitrary E...,exploit


In [97]:
from web3 import Web3

web3 = Web3(Web3.HTTPProvider("https://eth-mainnet.gateway.pokt.network/v1/lb/6266d6cdaa777e00391e0d29"))
addresses = []
opcodes = []

for index in range(len(sc_df)):
    address = sc_df.contract_address[index]
    tx_hash = sc_df.contract_creation_tx[index]
    code = web3.eth.getCode(Web3.toChecksumAddress(address)).hex()
    if code == '0x':
        for _ in range(10):
            try:
                tx = web3.eth.getTransaction(tx_hash)
                code = tx.input
                if code == b'':
                    raise Exception
                break
            except:
                continue

    if not code:
        continue
    # if (len(code) < 10):
    #
    #     print(code)
    opcode = EvmBytecode(code).disassemble()
    result = ''
    for j in opcode:
        result += f'{j.name} '
        if j.operand:
            result += f'0x{j.operand} '
    result = result[:-1]

    addresses.append(address)
    opcodes.append(result)




invalid instruction: PUSH21
invalid instruction: PUSH16


...


invalid instruction: PUSH20


...


invalid instruction: PUSH16
invalid instruction: PUSH29


...

invalid instruction: PUSH25
invalid instruction: PUSH20


0x60c0604052600080546001600160a01b0319908116734682812ccecc5ae94cbc874e91b02020a0e96f7917909155600180548216735c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f17905560028054821673c02aaa39b223fe8d0a0e5c4f27ead9083c756cc21790556003805490911673c18360217d8f7ab5e7c516566761ea12ce7f9d72179055680292b61b6c617865536004553480156100a157600080fd5b5033606081811b608052309081901b60a0526109e46100d8600039600060a8015260008181610636015261073201526109e46000f3fe6080604052600436106100435760003560e01c806310d1e85c1461004f5780632e88fb97146100715780634de260a2146100915780638da5cb5b1461009957600080fd5b3661004a57005b600080fd5b34801561005b57600080fd5b5061006f61006a3660046107e2565b6100e3565b005b34801561007d57600080fd5b5061006f61008c3660046107a1565b505050565b61006f61045c565b3480156100a557600080fd5b507f00000000000000000000000000000000000000000000000000000000000000006040516001600160a01b03909116815260200160405180910390f35b61010a60405180604001604052806006815260200165626f72726f7760d01b8152506104f3565b6002546000546004805460405163a905

invalid instruction: PUSH13


...


invalid instruction: PUSH19
invalid instruction: PUSH13


In [98]:
df = pd.DataFrame(list(zip(addresses, opcodes)), columns=['contract_address', 'opcodes'])

In [99]:
df

,contract_address,opcodes
0,0xc5ac25cfc2b8284e84ca47dad21cf1319f732c11,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
1,0x79dbe9bbde91a35fa8148a14084979a531fe57ea,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
2,0x3b1ea5b11d12452693f9bd290ac2100394e6850f,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
3,0x682dcf2f4a6e46c222927a54529b4965fb313bf2,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
4,0xbc48cd3265fd6d6cd413cd3e7082c27993baf8b2,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
...,...,...
111,0x016e20cad525bfb802ae3f12ecbcb64b5ad8410b,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
112,0xb5c827fdbbee6f6e9df3a5cb499aedf5927de1b8,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
113,0xe167cdaac8718b90c03cf2cb75dc976e24ee86d3,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
114,0x9682f31b3f572988f93c2b8382586ca26a866475,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...


In [100]:
df.to_csv('/home/venglov/Documents/mcd_data/data_v2/malicious_opcodes_116_gh.csv')

In [2]:
df = pd.read_csv('/home/venglov/Documents/mcd_data/data_v2/malicious_opcodes_116_gh.csv')

In [4]:
benign_df = pd.read_csv('/home/venglov/Documents/mcd_data/data/normal_df.csv').sample(frac=1).reset_index(drop=True)[:10000]

In [92]:


# for i in range(100):
#     opcodes = EvmBytecode(benign_df.creation_bytecode[i]).disassemble()
#     result = ''
#     for j in opcodes:
#         result += f'{j.name} '
#         if j.operand:
#             result += f'0x{j.operand} '
#     result = result[:-1]
#
#     if result != benign_df.decompiled_opcodes[i]:
#         test = benign_df.creation_bytecode[i]
#         # print(result)
#         # print()
#         # print(benign_df.decompiled_opcodes[i])
#         print('err')

invalid instruction: PUSH16
invalid instruction: PUSH14
invalid instruction: PUSH16
invalid instruction: PUSH31
invalid instruction: PUSH17
invalid instruction: PUSH20
invalid instruction: PUSH19
invalid instruction: PUSH26
invalid instruction: PUSH20
invalid instruction: PUSH31
invalid instruction: PUSH27
invalid instruction: PUSH22
invalid instruction: PUSH25
invalid instruction: PUSH14
invalid instruction: PUSH31
invalid instruction: PUSH16
invalid instruction: PUSH16
invalid instruction: PUSH22
invalid instruction: PUSH22
invalid instruction: PUSH21
invalid instruction: PUSH20
invalid instruction: PUSH31
invalid instruction: PUSH11
invalid instruction: PUSH31


In [82]:
# from evmdasm import EvmBytecode
# opcodes = EvmBytecode(test).disassemble()

In [87]:
# result = ''
# for i in opcodes:
#     result += f'{i.name} '
#     if i.operand:
#         result += f'0x{i.operand} '
# result = result[:-1]
#
# print(result)


PUSH1 0x60 PUSH1 0x40 SWAP1 DUP2 MSTORE PUSH1 0x03 DUP1 SLOAD PUSH1 0xa0 PUSH1 0x02 EXP PUSH1 0xff MUL NOT AND SWAP1 SSTORE PUSH1 0x00 PUSH1 0x04 SSTORE DUP1 MLOAD SWAP1 DUP2 ADD PUSH1 0x40 MSTORE PUSH1 0x07 DUP2 MSTORE PUSH32 0x44656d6574657200000000000000000000000000000000000000000000000000 PUSH1 0x20 DUP3 ADD MSTORE PUSH1 0x05 SWAP1 DUP1 MLOAD PUSH2 0x0060 SWAP3 SWAP2 PUSH1 0x20 ADD SWAP1 PUSH2 0x00d2 JUMP JUMPDEST POP PUSH1 0x40 DUP1 MLOAD SWAP1 DUP2 ADD PUSH1 0x40 MSTORE PUSH1 0x03 DUP2 MSTORE PUSH32 0x444d540000000000000000000000000000000000000000000000000000000000 PUSH1 0x20 DUP3 ADD MSTORE PUSH1 0x06 SWAP1 DUP1 MLOAD PUSH2 0x00a8 SWAP3 SWAP2 PUSH1 0x20 ADD SWAP1 PUSH2 0x00d2 JUMP JUMPDEST POP PUSH1 0x12 PUSH1 0x07 SSTORE JUMPDEST JUMPDEST PUSH1 0x03 DUP1 SLOAD PUSH1 0x01 PUSH1 0xa0 PUSH1 0x02 EXP SUB NOT AND CALLER PUSH1 0x01 PUSH1 0xa0 PUSH1 0x02 EXP SUB AND OR SWAP1 SSTORE JUMPDEST JUMPDEST PUSH2 0x0172 JUMP JUMPDEST DUP3 DUP1 SLOAD PUSH1 0x01 DUP2 PUSH1 0x01 AND ISZERO PUSH2

In [75]:
# total = ''
# for op in opcodes.as_string:
#     total += op.as_string + ' '

AttributeError: 'str' object has no attribute 'as_string'

In [71]:
# total

'PUSH1 PUSH1 SWAP1 DUP2 MSTORE PUSH1 DUP1 SLOAD PUSH1 PUSH1 EXP PUSH1 MUL NOT AND SWAP1 SSTORE PUSH1 PUSH1 SSTORE DUP1 MLOAD SWAP1 DUP2 ADD PUSH1 MSTORE PUSH1 DUP2 MSTORE PUSH32 PUSH1 DUP3 ADD MSTORE PUSH1 SWAP1 DUP1 MLOAD PUSH2 SWAP3 SWAP2 PUSH1 ADD SWAP1 PUSH2 JUMP JUMPDEST POP PUSH1 DUP1 MLOAD SWAP1 DUP2 ADD PUSH1 MSTORE PUSH1 DUP2 MSTORE PUSH32 PUSH1 DUP3 ADD MSTORE PUSH1 SWAP1 DUP1 MLOAD PUSH2 SWAP3 SWAP2 PUSH1 ADD SWAP1 PUSH2 JUMP JUMPDEST POP PUSH1 PUSH1 SSTORE JUMPDEST JUMPDEST PUSH1 DUP1 SLOAD PUSH1 PUSH1 PUSH1 EXP SUB NOT AND CALLER PUSH1 PUSH1 PUSH1 EXP SUB AND OR SWAP1 SSTORE JUMPDEST JUMPDEST PUSH2 JUMP JUMPDEST DUP3 DUP1 SLOAD PUSH1 DUP2 PUSH1 AND ISZERO PUSH2 MUL SUB AND PUSH1 SWAP1 DIV SWAP1 PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP1 PUSH1 ADD PUSH1 SWAP1 DIV DUP2 ADD SWAP3 DUP3 PUSH1 LT PUSH2 JUMPI DUP1 MLOAD PUSH1 NOT AND DUP4 DUP1 ADD OR DUP6 SSTORE PUSH2 JUMP JUMPDEST DUP3 DUP1 ADD PUSH1 ADD DUP6 SSTORE DUP3 ISZERO PUSH2 JUMPI SWAP2 DUP3 ADD JUMPDEST DUP3 DUP2 GT ISZERO PU

In [103]:
df

,contract_address,opcodes
0,0xc5ac25cfc2b8284e84ca47dad21cf1319f732c11,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
1,0x79dbe9bbde91a35fa8148a14084979a531fe57ea,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
2,0x3b1ea5b11d12452693f9bd290ac2100394e6850f,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
3,0x682dcf2f4a6e46c222927a54529b4965fb313bf2,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
4,0xbc48cd3265fd6d6cd413cd3e7082c27993baf8b2,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
...,...,...
111,0x016e20cad525bfb802ae3f12ecbcb64b5ad8410b,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
112,0xb5c827fdbbee6f6e9df3a5cb499aedf5927de1b8,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDA...
113,0xe167cdaac8718b90c03cf2cb75dc976e24ee86d3,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
114,0x9682f31b3f572988f93c2b8382586ca26a866475,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...


In [5]:
benign_df = benign_df[['contract_address', 'decompiled_opcodes']]

In [6]:
benign_df = benign_df.rename(columns={'decompiled_opcodes': 'opcodes'})

In [7]:
benign_df

,contract_address,opcodes
0,0xa8f851b1f2f115122169721b5895efeb7dfbb3b6,PUSH1 0xe0 PUSH1 0x40 MSTORE PUSH1 0x0b PUSH1 ...
1,0x9126b817ccca682beaa9f4eae734948ee1166af1,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
2,0xcca9daac56ac8acababa50b44145e821196b9830,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x40 MLOAD ...
3,0xaf6434ff6a3a1fe62473dfacccda043caff6793c,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x19 PUSH1 ...
4,0x9cf980e7529f94507fd6fc915a056097b769fa59,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...
...,...,...
9995,0xd2abaff547c45e97a2f90535b923a5094391ba9f,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x00 PUSH1 ...
9996,0xadb3138cbbbe11771c9d24d10cc2cf159b76d24d,PUSH2 0x0160 PUSH1 0x40 MSTORE CALLVALUE DUP1 ...
9997,0xa066620554af0d50703e5ee3c58c022c6ae5339d,PUSH2 0x0160 PUSH1 0x40 MSTORE PUSH1 0x08 DUP1...
9998,0x9aeed8919cf83313bbbe461fdd5a61e6d16db3e8,PUSH1 0xe0 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...


In [8]:
benign_df['malicious'] = False

In [9]:
df['malicious'] = True

In [10]:
opcodes_full_df = pd.concat([benign_df, df]).sample(frac=1).reset_index(drop=True)


In [11]:
opcodes_full_df

,contract_address,opcodes,malicious,Unnamed: 0
0,0x0016394e785f667bb472d4148ab88a5c391accdf,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False,NaN
1,0xbdc4dd4b2f62c446485a1d2e39569466a2fbd829,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False,NaN
2,0x3e9ba4be6affa324f188b10962317239e32ce7ad,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x00 PUSH1 ...,False,NaN
3,0xf5194c3325202f456c95c1cf0ca36f8475c1949f,PUSH1 0x60 PUSH2 0x375a PUSH2 0x0140 CODECOPY ...,False,NaN
4,0x5869d838be4f1efa10f2082efc957c79f7c3bf0b,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH3 0x00001b PU...,False,NaN
...,...,...,...,...
10111,0xc03ac1e9c6946fed9e36732cebbcd73b0c7a0637,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False,NaN
10112,0x069b0f5f2e6f1604896f30a401a3f4497583f201,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False,NaN
10113,0x6afbb9c196c727dd29a5771d25628c434e2303b1,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x07 DUP1 S...,False,NaN
10114,0xfa6b9ee08a94a26ea7b390346055cf52cad545fb,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x40 MLOAD ...,False,NaN


In [15]:
opcodes_full_df = opcodes_full_df[['contract_address', 'opcodes', 'malicious']]

In [16]:
opcodes_full_df

,contract_address,opcodes,malicious
0,0x0016394e785f667bb472d4148ab88a5c391accdf,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False
1,0xbdc4dd4b2f62c446485a1d2e39569466a2fbd829,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False
2,0x3e9ba4be6affa324f188b10962317239e32ce7ad,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x00 PUSH1 ...,False
3,0xf5194c3325202f456c95c1cf0ca36f8475c1949f,PUSH1 0x60 PUSH2 0x375a PUSH2 0x0140 CODECOPY ...,False
4,0x5869d838be4f1efa10f2082efc957c79f7c3bf0b,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH3 0x00001b PU...,False
...,...,...,...
10111,0xc03ac1e9c6946fed9e36732cebbcd73b0c7a0637,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False
10112,0x069b0f5f2e6f1604896f30a401a3f4497583f201,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False
10113,0x6afbb9c196c727dd29a5771d25628c434e2303b1,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x07 DUP1 S...,False
10114,0xfa6b9ee08a94a26ea7b390346055cf52cad545fb,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x40 MLOAD ...,False


In [17]:
opcodes_full_df.to_csv('/home/venglov/Documents/mcd_data/data_v2/opcodes_full_10k.csv')